In [23]:
%load_ext autoreload

import xarray as xr
import rasterio
import numpy as np
import osr
import arcpy

sys.path.append(r'C:\Users\Lewis\Documents\GitHub\tenement-tools\modules')
import cog

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [24]:
ds = xr.open_dataset(r"C:\Users\Lewis\Desktop\stac-c2a4d6ac3fee690c702ea70994c1e57b.nc")

In [35]:
aprx = arcpy.mp.ArcGISProject('CURRENT')   
m = aprx.activeMap
lyr = m.listLayers('mdr.crf')[0]

In [58]:
c = arcpy.Chart('derp')
c.type = 'line'
c.title = 'hi'
c.xAxis.field='x'
c.yAxis.field='y'
c.line.aggregatuib = 'median'
c.dataSource = lyr


RuntimeError: Invalid dataset: CIMPATH=map/mdr_crf.xml

In [34]:
c = arcpy.Chart('test')
c.dataSource = lyr.dataSource
#c.type = 'dataClock'
c

In [31]:
c.

'C:\\Users\\Lewis\\Documents\\GitHub\\tenement-tools\\arc\\project\\tenement-tools-project\\mdr.crf'

In [9]:
arcpy.charts.Chart.

<module 'arcpy.charts' from 'C:\\Program Files\\ArcGIS\\Pro\\Resources\\ArcPy\\arcpy\\charts.py'>

In [53]:
arcpy.env.addOutputsToMap = False

# load netcdf and mdr
mdr = arcpy.Raster(r"C:\Users\Lewis\Desktop\stac-f7bbc2d21a1f66550458c7c09616e602.nc", True)

# check if multidim
is_multidim = mdr.isMultidimensional

# get mdim info
mdr_info = mdr.mdinfo

# get variable names
mdr_vars = mdr.variables

# get times
#mdr_times = mdr.getDimensionValues('x' 'time')

# export as mdr
#mdr.save("in_memory/out4.crf")
mdr.save(r"C:\Users\Lewis\Desktop\yesy\out5.crf")

#mdr_out = arcpy.md.MakeMultidimensionalRasterLayer(
    #in_multidimensional_raster = r"C:\Users\Lewis\Desktop\stac-f7bbc2d21a1f66550458c7c09616e602.nc",
    #out_multidimensional_raster_layer=r"C:\Users\Lewis\Desktop\yesy\stacer.crf",
    #variables=["nbart_red", "nbart_green", "nbart_blue"])

In [56]:
tif_layer = arcpy.CopyRaster_management(in_raster=r"C:\Users\Lewis\Desktop\stac-f7bbc2d21a1f66550458c7c09616e602.nc", 
                                        out_rasterdataset=r"out6.crf")

In [1]:
aprx = arcpy.mp.ArcGISProject('CURRENT')   
m = aprx.activeMap


#m.addDataFromPath(mdr)


#m.addDataFromPath("in_memory/out.crf")

In [90]:
lyr = m.listLayers('mdr.crf')[0]